## 2.4 Importações e Carregamento

In [10]:
pip install geopandas

  Using cached geopandas-0.8.1-py2.py3-none-any.whl (962 kB)
  Using cached pyproj-3.0.0.post1-cp38-cp38-win_amd64.whl (14.4 MB)
  Using cached Fiona-1.8.18.tar.gz (1.3 MB)
Note: you may need to restart the kernel to use updated packages.


    ERROR: Command errored out with exit status 1:
     command: 'C:\ProgramData\Anaconda3\envs\enemmk\python.exe' -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\mkrom\\AppData\\Local\\Temp\\pip-install-u9y8i3_i\\fiona\\setup.py'"'"'; __file__='"'"'C:\\Users\\mkrom\\AppData\\Local\\Temp\\pip-install-u9y8i3_i\\fiona\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\mkrom\AppData\Local\Temp\pip-pip-egg-info-v4x4mllg'
         cwd: C:\Users\mkrom\AppData\Local\Temp\pip-install-u9y8i3_i\fiona\
    Complete output (1 lines):
    A GDAL API version must be specified. Provide a path to gdal-config using a GDAL_CONFIG environment variable or use a GDAL_VERSION environment variable.
    ----------------------------------------
ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full

In [11]:
# importações 
from matplotlib import pyplot as plt 
import matplotlib
import pandas as pd 
import seaborn as sns
import geopandas as gpd 

# Estilos
sns.set_theme()
sns.set_context("talk")

# Carregamentos 
# Carregando mapa
uf_br0 = gpd.read_file('Shapes/gadm36_BRA_0.shp')
uf_br1 = gpd.read_file('Shapes/gadm36_BRA_1.shp')
# Carregando microdados ENEM
raw_dados = pd.read_csv('https://raw.githubusercontent.com/MuriloKrominski/Data-Science_ENEM-2019/main/MICRODADOS_ENEM_2019_SAMPLE_43278.csv')

# Informações básicas
print(f'Registros: {len(raw_dados)}\nVariáveis: {len(raw_dados.columns)}')

ModuleNotFoundError: No module named 'geopandas'

## 2.5 Seleção de variáveis

In [ ]:
# Variaveis sociais 
variaveis_sociais = ['SG_UF_RESIDENCIA', 'NU_IDADE', 'TP_SEXO', 'TP_COR_RACA', 'TP_ST_CONCLUSAO', 'TP_ESCOLA', 'Q001', 'Q002']
variaveis_prova = ['TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT', 'TP_STATUS_REDACAO', 'NU_NOTA_REDACAO']

# idade mínima 
participantes_mais_de_15_anos = raw_dados.query('NU_IDADE >= 15 ')

# se o campo TP_PESENCA_CN == 1, significa que a pessoa estava presente na prova de ciencias da natureza
participantes_mais_de_15_anos = participantes_mais_de_15_anos.query('TP_PRESENCA_CN == 1 & TP_PRESENCA_MT == 1')

# para eliminar registros NaN, uso o dropna()
sifted_data = participantes_mais_de_15_anos[variaveis_sociais + variaveis_prova].dropna()
quantidade_alunos_fizeram_redacao = len(sifted_data)

As características computacionais das variáveis e do dataframe são:

In [ ]:
sifted_data.info()

# 4. Resultados e Discussões
## Quantos alunos com mais de 15 anos foram eliminados na prova redação

Como já foi dito anteriormente a prova de redação tem um caráter eliminatório, isso significa que pecar em uma das competências significa não poder entrar em uma universidade/faculdade ou ainda não conseguir tirar um comprovante de conclusão do ensino médio. Dito isso vamos dar uma breve olhada de como se distribuem a nota de redação de todos os candidatos:

In [ ]:
plt.figure(figsize=(12,7))
plt.title('Distribuição de frênquência das notas', pad=20, fontsize=18)
ax = sns.histplot(data = sifted_data, x='NU_NOTA_REDACAO', hue='TP_SEXO', palette='hot',bins=50)
ax.legend(['Feminino','Masculino'])
#ax.legend(['Não declarado','Branca','Preta','Parda','Amarela','Indígena'])
ax.set_ylabel('Candidatos', fontsize=14)
ax.set_xlabel('Nota Redação', fontsize=14)
ax = ax

Podemos verificar que existe uma diferença entre os sexos masculino e feminino. **A quantidade e a nota de mulheres que fazem o exame é maior que a dos homens**. Este é um resultado de mulheres estudarem mais que homens já é um resultado bem conhecido pelo IBGE [[8]](https://agenciadenoticias.ibge.gov.br/agencia-noticias/2012-agencia-de-noticias/noticias/20234-mulher-estuda-mais-trabalha-mais-e-ganha-menos-do-que-o-homem). 

Olhando para os participantes que tiveram nota zero na redação, podemos ver que mulheres também são mais eliminadas que homens. Mas é uma diferença esperada, na verdade este é um resultado sistematico tendo em vista que mais mulheres fazem o exame.

Podemos verificar a proporção de eliminados na prova de redação do enem 2019: 

In [ ]:
eliminados_na_redacao = len(sifted_data.query('NU_NOTA_REDACAO == 0.0'))
proporcao_eliminados_redacao = (len(sifted_data.query('NU_NOTA_REDACAO == 0.0')) / len(sifted_data)) * 100

total_M = len(sifted_data.query('TP_SEXO == "M"'))
total_F = len(sifted_data.query('TP_SEXO == "F"'))

quantidade_M = len(sifted_data.query('TP_SEXO == "M"').query('TP_STATUS_REDACAO != 1.0'))
quantidade_F = len(sifted_data.query('TP_SEXO == "F"').query('TP_STATUS_REDACAO != 1.0'))

print(f'Quantidade de candidatos que realizaram a prova: {quantidade_alunos_fizeram_redacao}')
print(f'Eliminados na redação: {eliminados_na_redacao}')
print(f'Proporção de eliminados: {proporcao_eliminados_redacao:.2f}%')
print(f'\nQuantidade de candidatos do sexo masculino: {quantidade_M}')
print(f'Quantidade de candidatos do sexo feminino: {quantidade_F}')

print(f'\nProporção Homens eliminados: {(quantidade_M / total_M) * 100: .2f}%')
print(f'Proporção Mulheres eliminadas: {(quantidade_F / total_F) * 100: .2f}%')

Dentre os 92420 candidatos com mais de 15 anos que fizeram a prova de redação do ENEM 2019, a quantidade de pessoas que zeraram a redação e portanto eliminados da prova foi de 2554. A proporção de candidatos eliminados neste quesito é de 2.76%. Também podemos confirmar numéricamente que mulheres fazem mais o exame do que os homens ao passo que, proporcionalmente, **mulheres foram menos desclassificadas na prova de redação nos microdados do ENEM 2019**. 

## Em quais competências que os candidatos mais foram eliminados? 

Nos concentrando nos candidatos que tiraram nota zero, que estão no extremo esquerdo do histograma apresentado anteriormente. Será retirado o rótulo "1: *Sem problemas*" pois indica que não houve nenhum problema de caráter eliminátorio da prova de redação e a maior parte dos candidatos encontra-se nesta situação como foi possível verificar graficamente. Colocá-los em comparação aos alunos que foram eliminados vai provocar distorções no gráfico. Então, analisando somente as competências dos candidatos que foram eliminados na prova de redação: 

In [ ]:
competencias = ('Anulada','Cópia Texto Motivador','Em Branco','Fuga ao tema','Não atendimento ao tipo textual','Texto insuficiente','Parte desconectada')

plt.figure(figsize=(15,6))
plt.title("Candidatos desclassificados no ENEM 2019", fontsize= 18, pad=20)
ax = sns.histplot(data = sifted_data.query('NU_NOTA_REDACAO == 0.0'),
                  x = 'TP_STATUS_REDACAO',
                  hue='TP_STATUS_REDACAO',
                  legend=True,
                  palette='plasma')

ax.set_ylabel('Candidatos', fontsize=14)
ax.set_xlabel('', fontsize=14)
ax.legend(competencias[::-1],loc='upper right')
ax.grid(True)
ax = ax

Esta é a frequência das competências não antendidas, dentre os candidatos que foram desclassificados. De forma quantitativa, podemos verificar que, dentre os candidatos que tem mais de 15 anos e foram eliminados na prova de redação temos: 

In [ ]:
candidatos_eliminados_redacao = sifted_data.query('NU_NOTA_REDACAO == 0.0')['TP_STATUS_REDACAO'].value_counts()
candidatos_eliminados_redacao_norm = sifted_data.query('NU_NOTA_REDACAO == 0.0')['TP_STATUS_REDACAO'].value_counts(normalize=True) * 100
candidatos_eliminados_redacao_norm.round(2)

Ou seja as porcentagens de cada desclassificado de acordo com as competências exigidas pelo ENEM, são: 

* Em Branco:    34.14%
* Fuga ao tema:    27.56%
* Cópia Texto Motivador:    19.26%
* Texto insuficiente:     6.50%
* Anulada     4.93%
* Parte desconectada:     4.03%
* Não atendimento ao tipo textual:    3.56%

Excluindo os 34.14% dos candidatos deixaram a prova em branco, pois não tem como saber como inferir qual seria o motivo da eliminação caso tivesse escrito algo na prova e se concentrando apenas nos outros casos, temos 

In [ ]:
print(f'Candidatos que não deixaram a prova em branco e foram eliminados: {candidatos_eliminados_redacao.iloc[1:].sum().round(2)}')
print(f'Proporção de candidatos que não deixaram a prova em branco e foram eliminados: {candidatos_eliminados_redacao_norm.iloc[1:].sum().round(2)}%')

65.86% dos candidatos que tem mais de 15 anos e realizaram a prova de redação, pertem ao grupo de candidatos que deixou a desejar em um dos critérios copiar texto motivador, fugir ao tema "*Democratização do acesso ao cinema no Brasil*", não atendimento ao tipo textual (Dissertativo argumentativo), texto insuficiente e parte desconectada.

Suponto este grupo de candidatos, com mais de 15 anos, segundo o INAF 2018 [[5]](https://acaoeducativa.org.br/wp-content/uploads/2018/08/Inaf2018_Relat%C3%B3rio-Resultados-Preliminares_v08Ago2018.pdf) em uma base de 796 avaliados que estão no ensino médio (ou seja mais de 15 anos), 42% estão na categoria "Alfabetização Elementar" cujos critérios de avaliação são: 

* Capaz de selecionar  uma  ou  mais  unidades  de  informação,  observando  certas  condições,  em  textos  diversos  de extensão média realizando pequenas inferências.

* Resolve problemas envolvendo operações básicas com números da ordem do milhar, que exigem certo grau de planejamento e controle (total de uma compra, troco, valor de prestações sem juros).
* Compara  ou  relaciona  informações  numéricas  ou  textuais  expressas  em  gráficos  ou  tabelas  simples, envolvendo situações de contexto cotidiano doméstico ou social.

* Reconhece  significado  de  representação  gráfica  de  direção  e/ou  sentido  de  uma  grandeza  (valores negativos, valores anteriores ou abaixo daquele tomado como referência).

**Então podemos fazer um comparativo entre as métricas do INAF (Índice Nacional de Analfabetismo Funcional), onde 42% dos 796 avaliados com mais de 15 anos estão classificados em "Alfabetização Elementar" e os 65.86% dos 2554 candidatos eliminados por não atender as relativamente parecidas com as especificações exigidas na prova de redação do ENEM**


## Qual a idade das pessoas desclassificadas na redação?

Outro ponto que é abordado na entrevista da Agência Brasil, e uma crítica feita pela professora Maria do Rosário Longo Mortatti é que muitos gestores não investem em educação para jovens e adultos porque esta faixa tem uma proporção menor se comparada com as outras. O grafico a seguir contruido a partir dos nossos dados reflete um pouco desta situação

In [ ]:
plt.figure(figsize=(12,7))
plt.title("Distribuição de idades dos candidados desclassificados na prova de redação", fontsize= 18, pad=20)
ax = sns.histplot(data = sifted_data.query('TP_STATUS_REDACAO != 1.0 '), x='NU_IDADE', hue='TP_SEXO', palette='hot')
ax.set_xlabel('Idade', fontsize=14)
ax.set_ylabel('Candidatos', fontsize=14)
ax = ax

A quantidade de alunos que foram eliminados na prova de redação é maior entre as pessoas menores de 20 anos. Mas em contraste do que é observado no estudo de taxa de analfabetos no brasil realizado pela PNAD "Quanto maior a idade, maior a taxa de analfabetismo", no ENEM se for usada a métrica da prova de redação para medir a taxa de analfabetismo a quantidade de pessoas mais velhas aparentemente são desclassificadas em uma proporção menor menor que os jovens. 

É importante salientar que o ENEM é uma prova que dura 4 horas e é realizada em dois domingos, tendo isso em mente é razoavavel pensar que mais jovens fazem o exame, portanto esta comparação entre a PNAD e o ENEM não é a mais adequada. Além de vários outros fatores.

Um questionamento totalmente valido é "Será que jovens e adultos estudam menos, porque estão em menor proporção? Ou será que estudam menos por falta de incentivos e oportunidades?". Neste tipo de análise, este questionamento é fundamental, pois se menos jovens e adultos chegam a se candidatar para fazer o exame nacional do ensino médio a inferência de que eles são em menor proporção estará enviesada. 

## Qual a região onde os candidatos foram mais eliminados?

Como já foi dito anteriormente, as regiões sul e sudeste tem a menor taxa de analfabetismo, seguido da região norte e centro-oeste e o local onde ocorre a maior taxa é na região é no nordeste brasileiro. Em partes este resultado pode ser observado também nos dados do ENEM.

In [ ]:
# criando o grupo de estados
dados_por_estado_eliminados_redacao = sifted_data.query('TP_STATUS_REDACAO != 1.0').groupby('SG_UF_RESIDENCIA').sum().sort_values('TP_PRESENCA_CN', ascending=False)

# separando dados por estado
dados_candidatos_por_estado = sifted_data.groupby('SG_UF_RESIDENCIA').sum().sort_values('TP_PRESENCA_CN', ascending=False) 
candidatos_normalizados_por_estado = (dados_por_estado_eliminados_redacao['TP_PRESENCA_CN'] /  dados_candidatos_por_estado['TP_PRESENCA_CN']).sort_values(ascending=False)*100

# Informações para o mapa
uf_br1['NAME_1'] = candidatos_normalizados_por_estado.index
uf_br1['NOTA_ENEM_NORMALIZADA'] = candidatos_normalizados_por_estado.values

ax = uf_br1.plot(figsize=(14,6),
                 column='NOTA_ENEM_NORMALIZADA',
                 cmap=matplotlib.cm.get_cmap('gnuplot'),
                 legend=True).set_axis_off()
ax = ax

Os lugares onde a cor tem um tom mais violeta representam uma quantidade menor de eliminados na prova de redação, já os estados onde a cor tem um tom mais alaranjado/amarelo tem uma quantidade de maior. Este resultado foi exposto de acordo com a proporção de candidatos de cada estado e é possível verificar que **a quantidade de pessoas que são desclassificadas na prova de redação é menor nas regiões sul e sudeste se comparadas com norte e nordeste.**




In [ ]:
# gerando o grafico
plt.figure(figsize=(15,6))
plt.title('Eliminados na prova de redação em proporção com nº de candidatos do estado', fontsize=18, pad=20)
ax = sns.barplot(y = candidatos_normalizados_por_estado[::-1],
                 x = candidatos_normalizados_por_estado.index[::-1],
                 palette='gnuplot')
ax.set_ylabel('(%) Candidatos Desclassificados', fontsize=14)
ax.set_xlabel('Sigla do Estado', fontsize=14)

ax = ax

Note que na região mais violeta, quem tem a menor quantidade de candidatos desclassificados pela redação, são majoritariamente estados da região sul e sudeste. Já **os estados que tem uma maior proporção de eliminados são Roraima, Ceará e Amazonas**.

Como já foi dito anteriormente, nas regiões sul e sudeste tem a menor taxa de analfabetismo, 3.3%, para pessoas com mais de 15 anos. No ENEM coincidentemente os estados que tem a menor taxa de desclassificação proporcionalmente ao numero de candidatos do estado são justamente os que fazem parte da região sul e sudeste.

**Podemos dizer estes resultados encontrados concordam em parte com resultados de analfabetismo por estados encontrado pela PNAD 2019. Digo em parte porque, novamente, o ENEM é realizado anualmente e nesta base de microdados temos 127 mil registros.**

## Qual a proporção entre a cor/raça das pessoas que foram eliminadas?

Vamos ver a distribuição da nota de redação dos candidatos:

In [ ]:
ax = sns.displot(data = sifted_data.query('TP_COR_RACA != [0,4,5]'), x='NU_NOTA_REDACAO',
                 col='TP_COR_RACA', hue='TP_COR_RACA', palette='Set1', bins=50,
                 alpha=0.9).set_xlabels('Nota').set_titles('')
ax = ax

O grafico acima mostra a distribuição de notas de acordo com a cor/raça. Em vermelho temos as pessoas que marcaram a cor/raça "branca", em azul as pessoas que marcaram "preta" e verde os candidatos que marcaram "parda". Optei por remover os candidatos que marcaram "Não responderam", "Amarelos" e "Indigena", porque as quantidades são muito pequenas nesta base de dados.

Analisando as pessoas que foram desclassificadas na prova de redação (no extremo esquerdo) de cada figura, vemos que a quantidade de pardos é a maior, seguida da quantidade de brancos e por fim negros, mas a quantidade de candidatos em cada uma das classificações também é diferentes. 

In [ ]:
total_candidatos = sifted_data.query('TP_COR_RACA == [1,2,3]').groupby('TP_COR_RACA').sum()['TP_PRESENCA_LC']
eliminados_redacao = sifted_data.query('TP_COR_RACA == [1,2,3] & NU_NOTA_REDACAO == 0.0').groupby('TP_COR_RACA').sum()['TP_PRESENCA_LC']
proporcao_de_eliminados = (eliminados_redacao/total_candidatos)

print((proporcao_de_eliminados*100).round(2))

Numericamente temos que a proporção de brancos eliminados é de 1.91%, pretos 3.24% e pardos 3.28%. Apesar desta classificação qualitativa entre pretos e pardos ser bem discutível, é inegavel que a maior parte das pessoas que zeram né de negros/pardos se comparado com a proporção de brancos. 

**Se levarmos em conta o dado do analfabetismo, dito anteriormente que brancos tem uma taxa menor de analfabetismo se comparado com pessoas negras, no ENEM isso também acontece com pessoas pessoas que foram eliminadas na prova de redação**.

Fazendo um comparativo com a primeira análise, onde ocorre que mais mulheres fazem o exame nacional e proporcionalmente são menos desclassificadas que homens na prova de redação, na analise de cor/raça isso não acontece. Porporcionalmente, mais pretos e pardos são desclassificados se comparados com os candidatos que marcaram cor/raça "branca" no questionario.

In [ ]:
abs((proporcao_de_eliminados.iloc[0] - proporcao_de_eliminados.iloc[1]) * 100).round(2)

De acordo com a pesquisa da PNAD,a taxa de analfabetismo entre negros é 3.6% maior em negros do que em brancos. Na prova de redação do ENEM essa diferença de proporções é de 1.33% se comprar apenas brancos e pretos.

## Qual a escolaridade dos pais do candidato que foi eliminado na prova de redação?

In [ ]:
# agrupando quem zerou na redação por escolaridade dos pais
escolaridade_pai_desclassificado = sifted_data.query('TP_STATUS_REDACAO != 1.0').groupby('Q001').sum()
escolaridade_mae_desclassificado = sifted_data.query('TP_STATUS_REDACAO != 1.0').groupby('Q002').sum()

In [ ]:
plt.figure(figsize=(12,5))
plt.title("Escolaridade do pai dos candidatos desclassificados no ENEM", fontsize= 18, pad=20)
ax = sns.barplot(data = escolaridade_pai_desclassificado,
                  y = 'TP_PRESENCA_CN',
                  x=escolaridade_pai_desclassificado.index,
                 
                  palette='plasma')

ax.set_ylabel('Candidatos', fontsize=14)
ax.set_xlabel('', fontsize=14)
ax = ax

Os rótulos de cada letra são: 
* A	Nunca estudou.
* B	Não completou a 4ª série/5º ano do Ensino Fundamental.
* C	Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental.
* D	Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio.
* E	Completou o Ensino Médio, mas não completou a Faculdade.
* F	Completou a Faculdade, mas não completou a Pós-graduação.
* G	Completou a Pós-graduação.
* H	Não sei.

Neste gráfico dois resultados me chamaram a atenção. Visualmente, a maior parte dos pais destes candidatos que não atigiram as competências exigidas pela avaliação de redação do ENEM, não completou a 4ª série/5º ano do Ensino Fundamental e a quarta categoria (H), onde o candidato informou "Não sei". Existem varias possibilidades pra não saber a escolaridade do pai, de qualquer forma, é um dado que me chamou a atenção.

Analisando a proporção desses resultados em ordem decrescente:

In [ ]:
total_candidatos = escolaridade_pai_desclassificado['TP_PRESENCA_CN'].sum()
escolaridade_pai_ordem = ((escolaridade_pai_desclassificado['TP_PRESENCA_CN'] / total_candidatos )  * 100).round(2).sort_values(ascending=False)
escolaridade_pai_ordem

A maior parte dos candidatos encontra-se na categoria **B** (*25.88%, Não completou a 4ª série/5º ano do Ensino Fundamental.*), em segundo temos a categoria **E** (*20.34%, Completou o Ensino Médio, mas não completou a Faculdade*), em terceiro temos categoria temos **C** onde o pai do candidato completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental.

Podemos concluir que dentre os candidatos que foram desclassificados na prova de redação o pai não frequentou a faculdade. Verificando a escolaridade da mãe temos: 

In [ ]:
plt.figure(figsize=(12,5))
plt.title("Escolaridade da mãe dos candidatos desclassificados no ENEM", fontsize= 18, pad=20)
ax = sns.barplot(data = escolaridade_mae_desclassificado,
                  y = 'TP_PRESENCA_CN',
                  x = escolaridade_mae_desclassificado.index,
                 
                  palette='plasma')

ax.set_ylabel('Candidatos', fontsize=14)
ax.set_xlabel('', fontsize=14)
ax = ax

* A	Nunca estudou.
* B	Não completou a 4ª série/5º ano do Ensino Fundamental.
* C	Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental.
* D	Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio.
* E	Completou o Ensino Médio, mas não completou a Faculdade.
* F	Completou a Faculdade, mas não completou a Pós-graduação.
* G	Completou a Pós-graduação.
* H	Não sei.

In [ ]:
total_candidatos = escolaridade_mae_desclassificado['TP_PRESENCA_CN'].sum()
escolaridade_mae_ordem = ((escolaridade_mae_desclassificado['TP_PRESENCA_CN'] / total_candidatos )  * 100).round(2).sort_values(ascending=False)
escolaridade_mae_ordem

Podemos notar que a primeira categoria **E**, onde a mãe do candidato completou o Ensino Médio, mas não completou a Faculdade. Como já foi dito anteriormente este é um resultado conhecido pelo IBGE [[8]](https://agenciadenoticias.ibge.gov.br/agencia-noticias/2012-agencia-de-noticias/noticias/20234-mulher-estuda-mais-trabalha-mais-e-ganha-menos-do-que-o-homem), mulheres estudam mais que homens e comparando com a proporção anterior podemos verificar isto. A segunda categoria vem o grupo onde a mãe não completou a 4ª série/5º ano do Ensino Fundamental seguida da categoria C em que a mãe completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental.

Podemos verificar se estes resultados se mantém em relação ao total de candidatos do ENEM:

In [ ]:
# agrupando quem zerou na redação por escolaridade dos pais
escolaridade_pai_desclassificado = sifted_data.groupby('Q001').sum()
escolaridade_mae_desclassificado = sifted_data.groupby('Q002').sum()

#escolaridade do pai 
total_candidatos = escolaridade_pai_desclassificado['TP_PRESENCA_CN'].sum()
escolaridade_pai_ordem = ((escolaridade_pai_desclassificado['TP_PRESENCA_CN'] / total_candidatos )  * 100).round(2).sort_values(ascending=False)
print('Escolaridade do Pai\n',escolaridade_pai_ordem)

#escolaridade da mãe
total_candidatos = escolaridade_mae_desclassificado['TP_PRESENCA_CN'].sum()
escolaridade_mae_ordem = ((escolaridade_mae_desclassificado['TP_PRESENCA_CN'] / total_candidatos )  * 100).round(2).sort_values(ascending=False)
print('\nEscolaridade do Mãe\n',escolaridade_mae_ordem)